### Configure Spark Memory Requirement

In [ ]:
%%configure -f
{"name": "Data Checking", "executorMemory": "2G", "driverMemory": "5G", "executorCores": 2, "numExecutors": 15}

### Pandas Display Configuration

In [2]:
%%local
import pandas
pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)
pandas.set_option('display.width', None)

### Create Connection to Azure Blob Storage

In [ ]:
spark._jsc.hadoopConfiguration().set('AZURE_BLOB', 'KEY')

### Path Variables for Varifying Data Structure

In [4]:
path="YOUR_CSV_PATH"

### Read CSV by Spark

In [5]:
def read_csv(path):
    return spark.read.csv(path)

In [6]:
def get_shape(df):
    print((df.count(), len(df.columns)))

### Compare DF and Dedup

In [7]:
def check_diff(df, column_list):
    if not column_list:
        column_list = []
    count = df.count()
    drop_duplicates_counts = df.dropDuplicates(column_list).count()
    if count != drop_duplicates_counts:
        print('Found Duplicates')
        print('Total Count: ' + str(count))
        print('Drop Duplicates Count: ' + str(drop_duplicates_counts))
        print('Duplicates Rows: ' + str(count - drop_duplicates_counts))
    else:
        print('No Duplicates Found')

In [8]:
from pyspark.sql.functions import collect_list, size, count, countDistinct, column

In [76]:
def not_null_checking(*argv):
    for arg in argv:
        i = create_column_name(arg)
        null_count = df.where(df[i].isNull()).count()
        if null_count > 0:
            print(str(i) + ': ' + str(null_count) + ' Found')
        else:
            print(i + ' Has No Null Value Found')

In [64]:
# df.count() == df.dropDuplicates(['_c1']).count()
def unikey_checking(column_list):
#     column_list = ['_c1']
    print('----------- Starting Column' + str(column_list[0]) + ' -----------')
    check_diff(df, column_list)
    grp = df.select(column_list).groupBy(column_list).count()
    #.groupBy(column_list).agg(count(column_list[0]).alias('count'))
    pandasDF = grp.filter('count != 1').toPandas()
    counter = 1
    for index, row in pandasDF.iterrows():
        print('----------- Get Row ' + str(counter) + ' -----------')
        df.where(column(column_list[0]) == row[0]).show(10)
# df.createOrReplaceTempView("stats")

In [77]:
def create_column_name(num):
    return '_c' + str(num)

### Checking CSV Dataset

In [ ]:
df = read_csv(path)
df.show(5)
not_null_checking(0,6,24)
unikey_checking(['_c0']) # Unikey, Primary Key Checking
unikey_checking(['_c24', '_c6']) # Compound Key Checking